# MECH 309: Assignment 2, Question 3

Cagri Arslan

January 30, 2025

*All work can be found on https://github.com/imported-canuck/MECH-309*

In [6]:
# Imports
import numpy as np 
from scipy import linalg

*Note:* I wrote all scripts by hand. Therefore, code might not be as concise/clean as a script produced by ChatGPT or taken from StackOverflow. Regardless, scripts should still be fully functional if all **assumptions** stated in docstrings are respected. I trust that this won't be penalized during grading. 

In [7]:
def forward_sub(A, b):
    """
    Solve the equation Ax = b for x, where A is a lower triangular matrix. Assumes a lower 
    triangular matrix A. Expects a that A is non-singular, and that the dimensions of 
    A and b are compatible (but checks these and throws an exception if not).
    
    Parameters:
    A (ndarray): A lower triangular matrix of shape (n, n).
    b (ndarray): A vector of shape (n, 1).
    
    Returns:
    x (ndarray): The solution vector of shape (n, 1).
    """
    n = A.shape[0]           # Size of matrix A
    if A.shape[1] != n:      # Check if A is square
        raise ValueError("Matrix A must be square.")
    if b.shape[0] != n:      # Check if dimensions of b are compatible
        raise ValueError("Vector b must have compatible dimensions with matrix A.")

    x = np.zeros((n, 1))           # Initialize solution vector x as a column vector
    
    x[0, 0] = b[0, 0] / A[0, 0]    # First element of x (actually redundant, kept for clarity)
    
    for i in range(n):       # Loop over each row
        if A[i, i] == 0:     # Check for singularity (zero diagonal element)
            raise ValueError("Matrix is singular.")
        
        # For each column of row i before the diagonal subtract the product of 
        # the entry A(i, j) and the corresponding entry of vector x from b[i]
        for j in range(i):
            b[i, 0] -= A[i, j] * x[j, 0] 

        # Finaly divide b[i] by the diagonal element A(i, i) to get x[i]
        x[i, 0] = b[i, 0] / A[i, i]  

    return x 


In [ ]:
def backward_sub(A, b):
    """
    Solve the equation Ax = b for x, where A is an upper triangular matrix. Assumes an upper 
    triangular matrix A. Expects a that A is non-singular, and that the dimensions of 
    A and b are compatible (but checks these and throws an exception if not).
    
    Parameters:
    A (ndarray): An upper triangular matrix of shape (n, n).
    b (ndarray): A vector of shape (n, 1).
    
    Returns:
    x (ndarray): The solution vector of shape (n, 1).
    """
    n = A.shape[0]               # Size of matrix A
    if A.shape[1] != n:          # Check if A is square
        raise ValueError("Matrix A must be square.")
    if b.shape[0] != n:          # Check if dimensions of b are compatible
        raise ValueError("Vector b must have compatible dimensions with matrix A.")

    x = np.zeros((n, 1))               # Initialize solution vector x
    
    x[-1, 0] = b[-1, 0] / A[-1, -1]    # First element of x (bottom-right; actually redundant, kept for clarity)
    
    for i in range(n - 1, -1 , -1):    # Loop over each row from bottom to top
        if A[i, i] == 0:               # Check for singularity (zero diagonal element)
            raise ValueError("Matrix is singular.")

        # For each column of row i after the diagonal subtract the product of 
        # the entry A(i, j) and the corresponding entry of vector x from b[i]    
        for j in range(i + 1, n):
            b[i, 0] -= A[i, j] * x[j, 0]

        # Finaly divide b[i] by the diagonal element A(i, i) to get x[i]
        x[i, 0] = b[i, 0] / A[i, i]
           
    return x


In [ ]:
def gaussian_elimination(A, b):
    """
    Solve the equation Ax = b for x using Gaussian elimination with partial pivoting. 
    Expects a that A is non-singular, and that the dimensions of A and b are 
    compatible (but checks these and throws an exception if not).
    
    Parameters:
    A (ndarray): An matrix of shape (n, n).
    b (ndarray): A vector of shape (n, 1).
    
    Returns:
    x (ndarray): The solution vector of shape (n, 1).
    """    
    tol = 1e-12                  # Tolerance to avoid floating point issues
    n = A.shape[0]               # Size of matrix A
    
    if A.shape[1] != n:          # Check if A is square
        raise ValueError("Matrix A must be square.")
    if b.shape[0] != n:          # Check if dimensions of b are compatible
        raise ValueError("Vector b must have compatible dimensions with matrix A.")

    for i in range(n - 1):       # Loop over all rows of A (apart from last, since A is already upper-triangular by then)

        p = i + np.argmax(np.abs(A[i:, i]))  # Partial pivoting: find the row with the largest element at the column position of the pivot

        if np.abs(A[p, i]) <= tol:           # If we can't find a row with a nonzero entry on the pivot point, matrix is singular 
                raise ValueError("Matrix is singular.")
            
        if p != i:               # If the current row is not the one with the greatest pivot element, do partial pivoting
                A[[i, p], :] = A[[p, i], :]  # Row swap current row "i" with row with greatest pivot element "p"
                b[[i, p], :] = b[[p, i], :]  # Row swap b vector the same way to maintain consistency

        for j in range(i + 1, n):        # For each subsequent row (starting at i + 1)...
            factor = A[j, i] / A[i, i]   # ... compute the factor that would the entry of it below the pivot
            A[j] = A[j] - factor * A[i]  # Eliminate the element of row j that is below the pivot
            b[j] = b[j] - factor * b[i]  # Apply the same operation on vector b

    # The matrix is now upper triangular, x can be solved for in O(n^2) time with backward substituiton
    return backward_sub(A, b)  


In [ ]:
def LU_factorization(A, b):
    """
    Solve the equation Ax = b for x using LU factorization with partial pivoting. 
    Expects a that A is non-singular, and that the dimensions of A and b are 
    compatible (but checks these and throws an exception if not).
    
    Parameters:
    A (ndarray): An matrix of shape (n, n).
    b (ndarray): A vector of shape (n, 1).
    
    Returns:
    x (ndarray): The solution vector of shape (n, 1).
    L (ndarray): The lower triangular matrix of shape (n, n).
    U (ndarray): The upper triangular matrix of shape (n, n).
    """    
    tol = 1e-12                  # Tolerance to avoid floating point issues
    n = A.shape[0]               # Size of matrix A

    if A.shape[1] != n:          # Check if A is square
        raise ValueError("Matrix A must be square.")
    if b.shape[0] != n:          # Check if dimensions of b are compatible
        raise ValueError("Vector b must have compatible dimensions with matrix A.")

    L = np.eye(n)                # Initialize L as identity matrix
    
    # Essentially apply gaussian elimination with partial pivoting to A, with
    # the added step of building the lower triangular matrix L along the way
    for i in range(n - 1):       

        p = i + np.argmax(np.abs(A[i:, i]))

        if np.abs(A[p, i]) <= tol:
                raise ValueError("Matrix is singular.")
            
        if p != i:
                A[[i, p], :] = A[[p, i], :]
                b[[i, p], :] = b[[p, i], :]
                L[[i, p], :i] = L[[p, i], :i]

        for j in range(i + 1, n): 
            factor = A[j, i] / A[i, i] 

            A[j] = A[j] - factor * A[i]
            L[j, i] = factor
            

    U = A.copy()

    y = forward_sub(L, b)
    x = backward_sub(U, y)

    return L, U, x
